In [47]:
#Siyuan Li
#last update: 2024/2/1


from sklearn.linear_model import LassoCV, ElasticNetCV,LogisticRegression
from sklearn.model_selection import cross_val_predict,train_test_split,KFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
from io import StringIO

In [43]:
file_path = 'E:\arrayexpress.txt'

with open(file_path, 'r') as file:
    lines = file.readlines()

sample_title = []
sample_geo_accession = []

for line in lines:
    if line.startswith("!Sample_title"):
        sample_title = line.split("\t")[0:]
    elif line.startswith("!Sample_geo_accession"):
        sample_geo_accession = line.split("\t")[0:]
        
data = pd.DataFrame({
    'Sample_title': sample_title[1:],
    'Sample_geo_accession': sample_geo_accession[1:]
})

print(data)


                                Sample_title Sample_geo_accession
0     "Genomic DNA from normal individual 1"         "GSM1183439"
1     "Genomic DNA from normal individual 2"         "GSM1183440"
2     "Genomic DNA from normal individual 3"         "GSM1183441"
3     "Genomic DNA from normal individual 4"         "GSM1183442"
4     "Genomic DNA from normal individual 5"         "GSM1183443"
..                                       ...                  ...
142    "Genomic DNA from adenoma patient 38"         "GSM1235154"
143    "Genomic DNA from adenoma patient 39"         "GSM1235155"
144    "Genomic DNA from adenoma patient 40"         "GSM1235156"
145    "Genomic DNA from adenoma patient 41"         "GSM1235157"
146  "Genomic DNA from adenoma patient 42"\n       "GSM1235158"\n

[147 rows x 2 columns]


In [51]:
data['status'] = data['Sample_title'].apply(lambda x: 0 if 'normal' in x else (1 if 'CRC' in x else None))

# delete'adenoma'
data = data[data['status'] != None]
# delete 'Sample_title'
data.drop(columns=['Sample_title'], inplace=True)

print(data.head())

  Sample_geo_accession  status
0         "GSM1183439"     0.0
1         "GSM1183440"     0.0
2         "GSM1183441"     0.0
3         "GSM1183442"     0.0
4         "GSM1183443"     0.0


In [ ]:

with open(file_path, 'r') as file:
    lines = file.readlines()


filtered_lines = [line for line in lines if not line.startswith('!')]

data2 = pd.read_csv(StringIO('\n'.join(filtered_lines)), sep='\t', header=None, index_col=0)
data2=data2.T

data2.head()

In [40]:
merged_data = pd.concat([data, data2],axis=1)
merged_data.head()

,Sample_title,Sample_geo_accession,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,...,rs7746156,rs798149,rs845016,rs877309,rs9292570,rs9363764,rs939290,rs951295,rs966367,rs9839873
0,"""Genomic DNA from normal individual 1""","""GSM1183439""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""Genomic DNA from normal individual 2""","""GSM1183440""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""Genomic DNA from normal individual 3""","""GSM1183441""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Genomic DNA from normal individual 4""","""GSM1183442""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Genomic DNA from normal individual 5""","""GSM1183443""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
selected_features = ['cg17187762', 'cg02964172']

X_train, X_test, y_train, y_test = train_test_split(merged_data[selected_features], merged_data['status'], test_size=0.2, random_state=42)


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("模型准确率：", accuracy)

